#Sistemas de Recomendação - 16/10/2021



1.   Salvar e carregar modelo de dados treinado;
2.   Inclusões de novas avaliações no arquivo de notas;
3. Modulando funções;
4. Comportamento das recomendações após novas avaliações;
5. Gerando e analisando arquivo XML com as predições;
6. Espaço para realização dos trabalhos.



In [1]:
#instalando a lib surprise
!pip install surprise

You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd


filmes = pd.read_csv('movies.csv')
notas = pd.read_csv('ratings.csv')
notas.columns = ['id_usuario','id_filme','nota_usuario_filme','dia_avaliacao']
filmes.columns = ['id_filme','titulo','genero']


#Buscando filmes assistidos e não assistidos de um usuário

In [3]:
filmes_assistidos = notas.query('id_usuario == 1')
filmes_nao_assistidos = filmes.set_index('id_filme').drop(filmes_assistidos.set_index('id_filme').index,errors='ignore')
filmes_nao_assistidos.head(5)

,titulo,genero
id_filme,,
2,Jumanji (1995),Adventure|Children|Fantasy
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
7,Sabrina (1995),Comedy|Romance
8,Tom and Huck (1995),Adventure|Children


In [4]:
#conferindo os filmes não assistidos
#ele compara o id do usuário e o id do filme e se não retornar nada, é porque está correto.
filmes_assistidos = notas.query('id_usuario == 584 and id_filme == 1237')
filmes_assistidos

,id_usuario,id_filme,nota_usuario_filme,dia_avaliacao


#Criando, treinando, salvando e carregando modelo

In [5]:
import surprise
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split 
import pickle

In [6]:
#reader = escala das notas, a estrutura da linha de entrada de dados -> id_u id_i avaliacao
reader = Reader(line_format='user item rating',rating_scale=(0.0,5.0))

In [7]:
#preparando os dados para a lib surprise, que deve ter somente as colunas id do usuario, id do filme e a nota
dados_surprise = notas[['id_usuario','id_filme','nota_usuario_filme']]
dados_preparados_treinamento = surprise.Dataset.load_from_df(dados_surprise,reader)

In [8]:
#temos os dados preparados em formato objeto surprise
#criando o modelo SVD
algo = surprise.SVDpp()

In [9]:
trainset, testset = train_test_split(dados_preparados_treinamento)

In [10]:
#para realizar o treinamento utiliza-se o algo.fit()
algo.fit(trainset)

In [11]:
pickle.dump(algo,open('treinado.pkl','wb'))

In [12]:
print('Predições dos filmes não assistidos do usuário 1')
dic=[]
#vamos pegar todos os filmes não assistidos, calcular a predição e salvar no dicionario
filmes_nao_assistidos = filmes_nao_assistidos.reset_index()
for index, row in filmes_nao_assistidos.iterrows():
  dic.append(algo.predict(1,row['id_filme']))
  

Predições dos filmes não assistidos do usuário 1


In [13]:
predicoes = pd.DataFrame.from_dict(dic)

In [14]:
predicoes.columns = ['id_usuario','id_filme','ref','predicao','destalhes']

In [15]:
recomendacoes = pd.merge(predicoes,filmes,how='inner',on='id_filme').sort_values('predicao',ascending=False)[['id_filme','titulo','predicao']]

In [16]:
#recomendacoes.head(50)

#Modulando funções para buscar filmes assistidos e não assistidos pelo id_filme

In [17]:
def busca_filmes_assistidos(id_usuario):
  print('Filmes assistidos do usuário %d' % id_usuario)
  filmes_assistidos = notas.query('id_usuario == %d' % id_usuario)
  return filmes_assistidos

In [18]:
def busca_filmes_nao_assistidos(id_usuario):
  filmes_assistidos = notas.query('id_usuario == %d' % id_usuario)
  id_filmes_assistidos = filmes_assistidos.set_index('id_filme').index
  filmes_nao_assistidos = filmes.set_index('id_filme').drop(id_filmes_assistidos,errors='ignore').reset_index()
  return filmes_nao_assistidos


In [19]:
def verifica_predicoes_filmes_nao_assistidos(id_usuario):
  print('Predicoes do usuário %d' % id_usuario)
  dic=[]
  #vamos pegar todos os filmes não assistidos, calcular a predição e salvar no dicionario
  filmes_predicoes = busca_filmes_nao_assistidos(id_usuario)
  for index, row in filmes_predicoes.iterrows():
    dic.append(algo.predict(id_usuario,row['id_filme']))
  return pd.DataFrame.from_dict(dic)

#Realizando predições

In [20]:
def apresenta_recomendacoes(id_usuario,numero_de_recomendacoes=10):
  filmes_assistidos = notas.query("id_usuario == %d " % id_usuario)
  if filmes_assistidos.empty:
    print("****O usuário com id %d ainda não avaliou nenhum filme****" % id_usuario)
    print("****Serão recomendados os filmes mais populares****")
    total_de_votos = notas['id_filme'].value_counts()
    filmes_backup = filmes.copy()
    filmes_backup['total_de_votos'] = total_de_votos
    notas_medias = notas.groupby('id_filme').mean()['nota_usuario_filme']
    filmes_backup['notas_medias'] = notas_medias
    return filmes_backup.sort_values('total_de_votos',ascending=False).head(numero_de_recomendacoes)

  predicoes = verifica_predicoes_filmes_nao_assistidos(id_usuario)
  predicoes.columns = ['id_usuario','id_filme','ref','predicao','destalhes']
  recomendacoes = pd.merge(predicoes,filmes,how='inner',on='id_filme').sort_values('predicao',ascending=False)[['id_filme','titulo','predicao']]
  return recomendacoes.head(numero_de_recomendacoes)

In [21]:
apresenta_recomendacoes(950,5)

****O usuário com id 950 ainda não avaliou nenhum filme****
****Serão recomendados os filmes mais populares****


,id_filme,titulo,genero,total_de_votos,notas_medias
356,412,"Age of Innocence, The (1993)",Drama,329.0,4.164134
318,360,I Love Trouble (1994),Action|Comedy,317.0,4.429022
296,338,Virtuosity (1995),Action|Sci-Fi|Thriller,307.0,4.197068
593,735,Cemetery Man (Dellamorte Dellamore) (1994),Horror,279.0,4.161290
2571,3439,Teenage Mutant Ninja Turtles II: The Secret of...,Action|Children|Fantasy,278.0,4.192446


In [22]:
apresenta_recomendacoes(584)

Predicoes do usuário 584


,id_filme,titulo,predicao
5802,33166,Crash (2004),5.0
7685,91529,"Dark Knight Rises, The (2012)",5.0
878,1262,"Great Escape, The (1963)",5.0
231,318,"Shawshank Redemption, The (1994)",5.0
976,1376,Star Trek IV: The Voyage Home (1986),5.0
3111,4306,Shrek (2001),5.0
4344,6539,Pirates of the Caribbean: The Curse of the Bla...,5.0
974,1374,Star Trek II: The Wrath of Khan (1982),5.0
5083,8368,Harry Potter and the Prisoner of Azkaban (2004),5.0
46,62,Mr. Holland's Opus (1995),5.0


#Inserindo dados no arquivo CSV

In [23]:
#CSV
import csv

In [24]:
with open('ratings.csv','a') as arquivo_csv:
  escrever = csv.writer(arquivo_csv,delimiter=',',lineterminator='\n')
  escrever.writerow([584,1223,5.0])

In [25]:
notas = pd.read_csv('ratings.csv')
notas.columns = ['id_usuario','id_filme','nota_usuario_filme','dia_avaliacao']

In [26]:
notas.tail()

,id_usuario,id_filme,nota_usuario_filme,dia_avaliacao
100832,610,168248,5.0,1.493850e+09
100833,610,168250,5.0,1.494273e+09
100834,610,168252,5.0,1.493846e+09
100835,610,170875,3.0,1.493846e+09
100836,584,1223,5.0,NaN


In [27]:
notas.tail()

,id_usuario,id_filme,nota_usuario_filme,dia_avaliacao
100832,610,168248,5.0,1.493850e+09
100833,610,168250,5.0,1.494273e+09
100834,610,168252,5.0,1.493846e+09
100835,610,170875,3.0,1.493846e+09
100836,584,1223,5.0,NaN


#Salvar predições em arquivo xml

In [28]:
import xml.etree.cElementTree as ET

In [29]:
def grava_arquivo_xml(dados,id_usuario):
    root = ET.Element("Catalogo")
    for index, row in dados.iterrows():
        doc = ET.SubElement(root, "Filme")
        ET.SubElement(doc, "Id_filme").text = str(row['id_filme'])
        ET.SubElement(doc, "Titulo").text = row['titulo']
    tree = ET.ElementTree(root)
    tree.write("filmes_us_"+str(id_usuario)+".xml")


In [30]:
grava_arquivo_xml(apresenta_recomendacoes(145,5),145)

Predicoes do usuário 145


#Função para realizar treinamento

In [31]:
#essa função vai criar o modelo e treinamento com os dados.
#escala menor é a nota mais baixa do conjunto de dados. -> 0.0
#escala maior é a nota mais alta do conjunto de dados. -> 0.5
def realiza_treinamento(dados, nome_modelo,escala_menor, escala_maior):
  if(nome_modelo == 'SVD') or (nome_modelo == 'svd'):
    print('Modelo escolhido foi o SVD')
    algo = surprise.SVD()
    print('Modelo gerado com sucesso')
  elif(nome_modelo == 'SVDpp') or (nome_modelo == 'svdpp') or (nome_modelo == 'SVDPP'):
    print('Modelo escolhido foi o SVDpp')
    algo = surprise.SVDpp()
    print('Modelo gerado com sucesso')

  reader = Reader(line_format='user item rating',rating_scale=(escala_menor,escala_maior))
  dados_surprise = dados[['id_usuario','id_filme','nota_usuario_filme']]
  dados_preparados_treinamento = surprise.Dataset.load_from_df(dados_surprise,reader)
  trainset, testset = train_test_split(dados_preparados_treinamento)
  algo.fit(trainset)
  print("Modelo "+nome_modelo+" treinado com sucesso")
  return algo


In [32]:
algo = realiza_treinamento(notas,'SVD',0.0,5.0)

Modelo escolhido foi o SVD
Modelo gerado com sucesso
Modelo SVD treinado com sucesso


#Criando menu para inserir avaliações, recarregar treinamento, apresentar recomendações

In [33]:
#função para salvar o modelo treinado
def salva_modelo(algo):
  print(algo)
  pickle.dump(algo,open('treinado.pkl','wb'))
  print("O comando dump foi executado sem erro")

In [34]:
def carrega_modelo():
  with open('treinado.pkl','rb') as modelo:
    algo = pickle.load(modelo)
  return algo

In [35]:
def verifica_predicoes_filmes_nao_assistidos(id_usuario,algo):
  print('Predicoes do usuário %d' % id_usuario)
  dic=[]
  #vamos pegar todos os filmes não assistidos, calcular a predição e salvar no dicionario
  filmes_predicoes = busca_filmes_nao_assistidos(id_usuario)
  for index, row in filmes_predicoes.iterrows():
    dic.append(algo.predict(id_usuario,row['id_filme']))
  return pd.DataFrame.from_dict(dic)

In [36]:
def apresenta_recomendacoes(id_usuario,notas,algo,filmes,numero_de_recomendacoes=10):
  filmes_assistidos = notas.query("id_usuario == %d " % id_usuario)
  if filmes_assistidos.empty:
    print("****O usuário com id %d ainda não avaliou nenhum filme****" % id_usuario)
    print("****Serão recomendados os filmes mais populares****")
    total_de_votos = notas['id_filme'].value_counts()
    filmes_backup = filmes.copy()
    filmes_backup['total_de_votos'] = total_de_votos
    notas_medias = notas.groupby('id_filme').mean()['nota_usuario_filme']
    filmes_backup['notas_medias'] = notas_medias
    return filmes_backup.sort_values('total_de_votos',ascending=False).head(numero_de_recomendacoes)

  predicoes = verifica_predicoes_filmes_nao_assistidos(id_usuario,algo)
  predicoes.columns = ['id_usuario','id_filme','ref','predicao','destalhes']
  recomendacoes = pd.merge(predicoes,filmes,how='inner',on='id_filme').sort_values('predicao',ascending=False)[['id_filme','titulo','predicao']]
  return recomendacoes.head(numero_de_recomendacoes)

In [37]:
def menu():
  continuar = 1
  while(continuar):
    continuar = int(input("0. Sair\n"+
                          "1. Inserir novas avaliações\n"+
                          "2. Verificar id de um novo usuário\n"+
                          "3. Realiza Treinamento\n"+
                          "4. Recomenda Filmes\n"
                          )
                    )
    if(continuar == 1):
      usuario = input("Digite o id do usuário: ")
      filme = input("Digite o id do filme: ")
      nota = input("Digite a nota do filme: ")

      with open('ratings.csv','a') as arquivo_csv:
        escrever = csv.writer(arquivo_csv,delimiter=',',lineterminator='\n')
        escrever.writerow([usuario,filme,nota])

    if(continuar == 2):
      notas = pd.read_csv('ratings.csv')
      notas.columns = ['id_usuario','id_filme','nota_usuario_filme','dia_avaliacao']
      novo_usuario = notas['id_usuario'].max()+1
      print("O próximo ID de usuário livre eh: %d" % novo_usuario)

    if(continuar == 3):
      notas = pd.read_csv('ratings.csv')
      notas.columns = ['id_usuario','id_filme','nota_usuario_filme','dia_avaliacao']
      modelo = input("Digite o nome do modelo: ")
      escala_menor = float(input("Digite o valor da nota mais baixa: "))
      escala_maior = float(input("Digite o valor da nota mais alta: "))
      algo = realiza_treinamento(notas,modelo,escala_menor,escala_maior)
      salva_modelo(algo)

    if(continuar == 4):
      notas = pd.read_csv('ratings.csv')
      notas.columns = ['id_usuario','id_filme','nota_usuario_filme','dia_avaliacao']
      algo = carrega_modelo()
      id_usuario = int(input("Recomendar filmes para: "))
      numero_de_recomendacoes = int(input("Digite o número de filmes a recomendar: "))
      recomendacoes = apresenta_recomendacoes(id_usuario,notas,algo,filmes,numero_de_recomendacoes)
      grava_arquivo_xml(recomendacoes,id_usuario)
      print(recomendacoes[['id_filme','titulo']])
      

In [38]:
menu()

ValueError: invalid literal for int() with base 10: ''

In [ ]:
notas = pd.read_csv('ratings.csv')
notas.columns = ['id_usuario','id_filme','nota_usuario_filme','dia_avaliacao']
notas.tail(5)

,id_usuario,id_filme,nota_usuario_filme,dia_avaliacao
100837,584,1223,5.0,NaN
100838,584,168252,4.3,NaN
100839,584,1197,2.5,NaN
100840,584,318,5.0,NaN
100841,790,3439,5.0,NaN


#Verificar recomendações após novas avaliações

#Base de dados escolhida

